In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl huggingface_hub gradio openai

In [2]:
%%capture
%pip install --upgrade langchain

In [3]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    logging,
    BitsAndBytesConfig
)
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import requests
import pandas as pd
import gradio as gr

In [4]:
adapter_model = "naqib3110/llama-2-7b-chat-whazzat"
base_model = "meta-llama/Llama-2-7b-chat-hf"
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(base_model,
    quantization_config=quant_config,
    device_map={"": 0},
    token='YOUR_HF_ACCESS_TOKEN')
model.config.use_cache = False
model.config.pretraining_tp = 1
model.load_adapter(adapter_model)

pipeline = pipeline(
    "text-generation",
    model="naqib3110/llama-2-7b-chat-whazzat",
    tokenizer="naqib3110/llama-2-7b-chat-whazzat",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=250,
    do_sample=True,
    top_k=10,
    token='YOUR_HF_ACCESS_TOKEN'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
template = """
           You are an expert product recommender,
           You can generate product recommendations based on simple description of users' need.
           Recommend products that are as relevant as possible .
           Recommend only 5 products.
           Do not generate additional prompt as a user
           Only generate 1 array for the asked usecase
           Return nothing but an array of 5 items. Leave out any extra words that have nothing to do with the product names.

           example,
           USER: something to clean my house
           PRODUCT RECOMMENDER: ['Vacuum Cleaner','Mop and Bucket','Broom and Dustpan','Steam Cleaner','Robot Vacuum'].

           USER: {query}
           PRODUCT RECOMMENDER:
           """
llm = HuggingFacePipeline(pipeline=pipeline,model_kwargs={'temperature':0.6})
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [6]:
def cleanGenOutput(result):
  cleaned_str = result.replace("'", " ").replace("[", " ").replace("]"," ").split(',')
  stripped = [s.strip() for s in cleaned_str]
  print(stripped)
  url = "https://axesso-axesso-amazon-data-service-v1.p.rapidapi.com/amz/amazon-search-by-keyword-asin"
  headers = {
    "X-RapidAPI-Key": "YOUR_RAPIDAPI_ACCESS_TOKEN",
    "X-RapidAPI-Host": "axesso-axesso-amazon-data-service-v1.p.rapidapi.com"
  }
  df = pd.DataFrame()
  for i in stripped:
    print(i)
    querystring = {"domainCode":"ca","keyword":i,"page":"1","excludeSponsored":"false","sortBy":"relevanceblender","withCache":"true"}
    response = requests.get(url, headers=headers, params=querystring)
    productDetails = response.json()['searchProductDetails']
    # productDetails = response['searchProductDetails']
    df1=pd.DataFrame(productDetails)
    df1['productRating']=df1['productRating'].str.split(expand=True)[0]
    df1['dpUrl']="https://amazon.ca" + df1['dpUrl']
    # print(df.keys())
    col_list = ['productDescription', 'countReview', 'imgUrl', 'price', 'retailPrice', 'productRating', 'dpUrl']
    temp = df1[col_list]
    # df = df.append(temp)
    df = pd.concat([df, temp.head(15)], ignore_index=False)

  df = df.head(50)
  df.sort_values(by=['countReview', 'productRating'], inplace=True, ascending=False)

  df.sort_values(by=['price'], inplace=True)
  df = df.reset_index()
  df = df.drop(columns=['index'])
  return df

In [7]:
def gen(query):
    result = llm_chain.run(query)
    cleaned_str = result.split('.')[0].split('\n')[0]
    print(cleaned_str)

    return cleaned_str, cleanGenOutput(cleaned_str)

In [8]:
def outputdf(text=None):
  gen_output, response = gen(text)
  result = response.values.tolist()
  # flat_list = [item for sublist in result for item in sublist]
  html_output = f"<div>Result is showing for {gen_output} </div><br/><div style='display: flex; flex-direction: column;'>"

  for item in result:
      html_output += f"""
      <div style='display: flex; align-items: center; margin-bottom: 10px;'>
          <div style='width: 100px;background: #fff;margin: 10px;display: flex;align-items: center;justify-content: center;'><img src='{item[2]}' style='height: 5rem; margin-right: 10px;'/></div>
          <div>
              <b>Name:</b> {item[0]}<br>
              <b>Price:</b> ${item[3]}<br>
              <b>Rating:</b> {item[5]}/5<br>
              <a href='{item[6]}'>Amazon Link</a>
          </div>
      </div>
      """

  return html_output


In [9]:
import gradio as gr

css = ".output_image {height: 5rem !important; width: 10% !important;}"

def create_interface():
    with gr.Blocks(css=css) as demo:
        with gr.Row():
            user_input = gr.Textbox(label="Enter text here")
            submit_button = gr.Button("Submit")

        output_html = gr.HTML()

        submit_button.click(fn=outputdf, inputs=user_input, outputs=output_html)

    return demo

demo = create_interface()
demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://bdc2bcd447b926a812.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


 ['Scented Candle', 'Aromatherapy Diffuser', 'Weighted Blanket', 'Relaxing Eye Mask', 'Comfortable Pillow']
['Scented Candle', 'Aromatherapy Diffuser', 'Weighted Blanket', 'Relaxing Eye Mask', 'Comfortable Pillow']
Scented Candle
Aromatherapy Diffuser
Weighted Blanket
Relaxing Eye Mask
Comfortable Pillow
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://bdc2bcd447b926a812.gradio.live
